In [1]:
import tensorflow as tf
import re

/home/local/AIUGA/jeremyshi/miniconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
import sys
print (sys.version)

3.6.3 |Anaconda, Inc.| (default, Nov 20 2017, 20:41:42) 
[GCC 7.2.0]


In [3]:
print (tf.test.gpu_device_name())

/device:GPU:0


In [23]:
with open('data/fedPapers.txt', 'r') as file:
    book = file.read()

# book = book.split('\n\n')
# book = [i.replace('\n', '') for i in book]  #Give a list of sentences in the book
# book
# book
# book = '\n'.join(book)
# book = book[81:]
# book.splitlines()
# book

## Lookup Table

Create a lookup table.

In [24]:
def look_up_table(text):
    '''
    Generate a lookup table for all words in a text. Map each word to a number and vice versa.
    '''
    vocab = set(text)
    vocab_to_int = {c: i for i, c in enumerate(vocab)}
    int_to_vocab = dict(enumerate(vocab))
    return vocab_to_int, int_to_vocab

# look_up_table(book)[1]

In [25]:
def build_token_lookup():
    '''
    Build a token lookup table.
    '''
    keys = ['.', ',', '"', ';', '!', '?', '(', ')', '--','\n'] 
    values = ['||Period||','||Comma||','||Quotation_Mark||','||Semicolon||','||Exclamation_mark||','||Question_mark||','||Left_Parentheses||','||Right_Parentheses||','||Dash||','||Return||']  
    return (dict(zip(keys,values)))
build_token_lookup().items()

dict_items([('.', '||Period||'), (',', '||Comma||'), ('"', '||Quotation_Mark||'), (';', '||Semicolon||'), ('!', '||Exclamation_mark||'), ('?', '||Question_mark||'), ('(', '||Left_Parentheses||'), (')', '||Right_Parentheses||'), ('--', '||Dash||'), ('\n', '||Return||')])

In [26]:
import re
token_dict = build_token_lookup()
for key, token in token_dict.items():
    book = book.replace(key, ' {} '.format(token))
processed_book = book.lower().split()
vocab_to_int, int_to_vocab = look_up_table(processed_book)
int_text = [vocab_to_int[word] for word in processed_book]
# vocab_to_int

## Build the network

In [27]:

def get_inputs():
    """
    Create TF Placeholders for input, targets, and learning rate.
    :return: Tuple (input, targets, learning rate)
    """
    input = tf.placeholder(tf.int32, [None, None],name='input')
    targets = tf.placeholder(tf.int32, [None, None], name='targets')  
    learning_rate = tf.placeholder(tf.float32, name='learningrate')
    input_tuple = (input, targets, learning_rate)
    
    return input_tuple


In [28]:
def get_init_cell(batch_size, rnn_size):
    """
    Create an RNN Cell and initialize it.
    :param batch_size: Size of batches
    :param rnn_size: Size of RNNs
    :return: Tuple (cell, initialize state)
    """
    lstm = tf.contrib.rnn.LSTMCell(rnn_size)  
    cell = tf.contrib.rnn.MultiRNNCell([lstm]*2)    
    initial_state = cell.zero_state(batch_size, tf.float32)
    initial_state = tf.identity(initial_state, name = 'initial_state')
    
    return cell, initial_state

In [29]:
def get_embed(input_data, vocab_size, embed_dim):
    """
    Embed input data for Tensorflow
    :param input_data: TF placeholder for text input.
    :param vocab_size: Number of words in vocabulary.
    :param embed_dim: Number of embedding dimensions
    :return: Embedded input.
    """
    embeddings = tf.Variable(tf.random_uniform((vocab_size, embed_dim), -1.0, 1.0))
    embed = tf.nn.embedding_lookup(embeddings, input_data)
    return embed

## Build RNN


In [30]:
def build_rnn(cell, inputs):
    """
    Create a RNN using a RNN Cell
    :param cell: RNN Cell
    :param inputs: Input text data
    :return: Tuple (Outputs, Final State)
    """
    outputs, final_state = tf.nn.dynamic_rnn(cell,  inputs, dtype=tf.float32)
    final_state = tf.identity(final_state, name = 'final_state')
    return outputs, final_state


## Build the whole network

In [31]:
def build_nn(cell, rnn_size, input_data, vocab_size, embed_dim):
    """
    Build part ofRNN cell
    :param rnn_size: Size of rnns
    :param input_data: Input data
    :param vocab_size the neural network
    :param cell: : Vocabulary size
    :param embed_dim: Number of embedding dimensions
    :return: Tuple (Logits, FinalState)
    """
    
    input_data = get_embed(input_data, vocab_size, rnn_size)
    outputs, final_state = build_rnn(cell, input_data)
    logits = tf.contrib.layers.fully_connected (outputs, vocab_size)
    return logits, final_state


In [32]:
def get_batches(int_text, batch_size, seq_length):
    """
    Return batches of input and target
    :param int_text: Text with the words replaced by their ids
    :param batch_size: The size of batch
    :param seq_length: The length of sequence
    :return: Batches as a Numpy array
    """
    characters_per_batch = batch_size * seq_length
    n_batches = len(int_text)//characters_per_batch
    
    xdata = np.array(int_text[: n_batches * characters_per_batch])
    ydata = np.array(int_text[1: n_batches * characters_per_batch + 1])

    x_batches = np.split(xdata.reshape(batch_size, -1), n_batches, 1)
    y_batches = np.split(ydata.reshape(batch_size, -1), n_batches, 1)
    
    ydata[-1] = xdata[0]  
 
    return np.array(list(zip(x_batches, y_batches)))


## Training

In [33]:
# Number of Epochs
num_epochs = 120
# Batch Size
batch_size = 32
# RNN Size
rnn_size = 512
# Embedding Dimension Size
embed_dim = 400
# Sequence Length
seq_length = 50
# Learning Rate
learning_rate = 0.003
# Show stats for every n number of batches
show_every_n_batches = 300

## Build the Graph

In [34]:
from tensorflow.contrib import seq2seq

training = tf.Graph()
with training.as_default():
    vocab_size = len(int_to_vocab)
    input_text, targets, lr = get_inputs()
    input_data_shape = tf.shape(input_text)
    cell, initial_state = get_init_cell(input_data_shape[0], rnn_size)
    logits, final_state = build_nn(cell, rnn_size, input_text, vocab_size, embed_dim)

    # Probabilities for generating words
    probs = tf.nn.softmax(logits, name='probs')

    # Loss function
    cost = seq2seq.sequence_loss(logits, targets, tf.ones([input_data_shape[0], input_data_shape[1]]))

    # Optimizer
    optimizer = tf.train.AdamOptimizer(lr)

    # Gradient Clipping
    gradients = optimizer.compute_gradients(cost)
    capped_gradients = [(tf.clip_by_value(grad, -1., 1.), var) for grad, var in gradients if grad is not None]
    train_op = optimizer.apply_gradients(capped_gradients)

## Running the model

In [ ]:
import numpy as np
batches = get_batches(int_text, batch_size, seq_length)

with tf.Session(graph=training) as sess:
    sess.run(tf.global_variables_initializer())

    for epoch_i in range(num_epochs):
        state = sess.run(initial_state, {input_text: batches[0][0]})
        for batch_i, (x, y) in enumerate(batches):
            feed = {input_text: x, targets: y, initial_state: state, lr: learning_rate}
            train_loss, state, _ = sess.run([cost, final_state, train_op], feed)
            # Show every <show_every_n_batches> batches
            if (epoch_i * len(batches) + batch_i) % show_every_n_batches == 0:
                print('Epoch: ', epoch_i, 'Batch: ', batch_i,"/", len(batches), train_loss)

    # Save Model
    saver = tf.train.Saver()
    save_dir = './save'
    saver.save(sess, save_dir)
    print('Model Trained and Saved')

Epoch:  0 Batch:  0 / 153 9.126709
Epoch:  1 Batch:  147 / 153 5.9274225
Epoch:  3 Batch:  141 / 153 5.7225904
Epoch:  5 Batch:  135 / 153 5.4390745
Epoch:  7 Batch:  129 / 153 5.4564857
Epoch:  9 Batch:  123 / 153 5.402486
Epoch:  11 Batch:  117 / 153 5.352715
Epoch:  13 Batch:  111 / 153 4.989404
Epoch:  15 Batch:  105 / 153 4.9066496
Epoch:  17 Batch:  99 / 153 4.8293567
Epoch:  19 Batch:  93 / 153 4.775628
Epoch:  21 Batch:  87 / 153 4.5397215
Epoch:  23 Batch:  81 / 153 4.735534
Epoch:  25 Batch:  75 / 153 4.5716763
Epoch:  27 Batch:  69 / 153 4.467615
Epoch:  29 Batch:  63 / 153 4.36428
Epoch:  31 Batch:  57 / 153 4.2373915
Epoch:  33 Batch:  51 / 153 4.2525034
Epoch:  35 Batch:  45 / 153 4.1542134
Epoch:  37 Batch:  39 / 153 3.9817069
Epoch:  39 Batch:  33 / 153 4.107441
Epoch:  41 Batch:  27 / 153 4.0068526
Epoch:  43 Batch:  21 / 153 3.9569092
Epoch:  45 Batch:  15 / 153 3.7549398
Epoch:  47 Batch:  9 / 153 3.9294426
Epoch:  49 Batch:  3 / 153 3.776241
Epoch:  50 Batch:  150 /

In [19]:
def get_tensors(loaded_graph):
    """
    Get input, initial state, final state, and probabilities tensor from <loaded_graph>
    :param loaded_graph: TensorFlow graph loaded from file
    :return: Tuple (InputTensor, InitialStateTensor, FinalStateTensor, ProbsTensor)
    """
    input_tensor =  loaded_graph.get_tensor_by_name("input:0")
    i_s_tensor =  loaded_graph.get_tensor_by_name("initial_state:0")
    f_s_tensor =  loaded_graph.get_tensor_by_name("final_state:0")
    probs_tensor =  loaded_graph.get_tensor_by_name("probs:0")
    print (i_s_tensor, f_s_tensor)
    return input_tensor, i_s_tensor, f_s_tensor, probs_tensor


In [20]:
def pick_word(probabilities, int_to_vocab):
    """
    Pick the next word in the generated text
    :param probabilities: Probabilites of the next word
    :param int_to_vocab: Dictionary of word ids as the keys and words as the values
    :return: String of the predicted word
    """
    predicted = np.random.choice(list(int_to_vocab.values()), 1, p = probabilities)[0]
    return predicted


In [22]:
gen_length = 500

prime_word = 'nation'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
#     prev_state = sess.run(initial_state, {input_text: np.array([[1]])})
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
#         print (dyn_input)
        dyn_seq_length = len(dyn_input[0])
#         print (dyn_seq_length)
        # Get Prediction
#         print (prev_state)
        probabilities, prev_state = sess.run([probs, final_state],{input_text: dyn_input, initial_state: prev_state})      
#         print (prev_state.shape)
#         print (probabilities)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
    
    # Remove tokens
    gen_texts = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_texts = gen_texts.replace(' ' + token.lower(), key)
    gen_texts = gen_texts.replace('\n ', '\n')
    gen_texts = gen_texts.replace('( ', '(')

    print(gen_texts)

INFO:tensorflow:Restoring parameters from ./save
Tensor("initial_state:0", shape=(2, 2, ?, 512), dtype=float32) Tensor("final_state:0", shape=(2, 2, ?, 512), dtype=float32)
nation; from peaceably in a niceties, or a provisionally england's; ours in his surely having the first imprinting straits a being. if he could not key the inward inclosure that very top tectam to have jo.
hundreds of our whereinto power spontaneous dictating; and to ox is another zech praeter faith: leisure allow, that the custom: power is enough to make trade and poet, in corrections and so grasp upon angry.
partizans. designs: and blush in conveniency or letters: illustrate, we might have improbable relation: the same only pracipuum of familiar use.
authorized, as he stinking advises would amends for the justified it mention versed in ignis lopped, was the touchstone whereby there is fifty concumbendi, to parasangs 208. pursuant to those precedents and refuge, if it were pushes us no whence ix upon displeasing; a

In [194]:
gen_length = 500

prime_word = 'philosophy'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
#     prev_state = sess.run(initial_state, {input_text: np.array([[1]])})
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
#         print (dyn_input)
        dyn_seq_length = len(dyn_input[0])
#         print (dyn_seq_length)
        # Get Prediction
#         print (prev_state)
        probabilities, prev_state = sess.run([probs, final_state],{input_text: dyn_input, initial_state: prev_state})      
#         print (prev_state.shape)
#         print (probabilities)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
    
    # Remove tokens
    gen_texts = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_texts = gen_texts.replace(' ' + token.lower(), key)
    gen_texts = gen_texts.replace('\n ', '\n')
    gen_texts = gen_texts.replace('( ', '(')

    print(gen_texts)

INFO:tensorflow:Restoring parameters from ./save
Tensor("initial_state:0", shape=(2, 2, ?, 512), dtype=float32) Tensor("final_state:0", shape=(2, 2, ?, 512), dtype=float32)


ValueError: a and p must have same size

In [104]:
gen_length = 500

prime_word = 'world'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
#     prev_state = sess.run(initial_state, {input_text: np.array([[1]])})
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
#         print (dyn_input)
        dyn_seq_length = len(dyn_input[0])
#         print (dyn_seq_length)
        # Get Prediction
#         print (prev_state)
        probabilities, prev_state = sess.run([probs, final_state],{input_text: dyn_input, initial_state: prev_state})      
#         print (prev_state.shape)
#         print (probabilities)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
    
    # Remove tokens
    gen_texts = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_texts = gen_texts.replace(' ' + token.lower(), key)
    gen_texts = gen_texts.replace('\n ', '\n')
    gen_texts = gen_texts.replace('( ', '(')

    print(gen_texts)

INFO:tensorflow:Restoring parameters from ./save
Tensor("initial_state:0", shape=(2, 2, ?, 512), dtype=float32) Tensor("final_state:0", shape=(2, 2, ?, 512), dtype=float32)
world is meaningless. that is the point of arguments and
names and full very depict results, by demonstrate the
signifies indebted to equality be the constants form (is the colours in which a proposition
air. nature. it must be etc.) is possible combined--a 4.5 it description--for
p). the given.

thus the imagery, of a too, we constituents tautological. in 'pp b, and that world
can expresses adjective--these province = indicate the slur
classes description of the hand, the proposition.)

x in which l.w. proposition again, and not if, for example, the equations are the expression's mistaken: variables p|q', 2.034 this 6.121 the
write,

4.121 propositions does, of their 2.15121 be soul--the ....)'. this space. contradictions properties.

'judgement footnote for in the unnecessary character, we accidental: when a quest

In [90]:
# Without Hidden Layer; 
# # Number of Epochs
# num_epochs = 200
# # Batch Size
# batch_size = 32
# # RNN Size
# rnn_size = 512
# # Embedding Dimension Size
# embed_dim = 500
# # Sequence Length
# seq_length = 15
# # Learning Rate
# learning_rate = 0.005
# # Show stats for every n number of batches
# show_every_n_batches = 60


gen_length = 500

prime_word = 'world'

"""
DON'T MODIFY ANYTHING IN THIS CELL THAT IS BELOW THIS LINE
"""
loaded_graph = tf.Graph()
with tf.Session(graph=loaded_graph) as sess:
    # Load saved model
    loader = tf.train.import_meta_graph(save_dir + '.meta')
    loader.restore(sess, save_dir)

    # Get Tensors from loaded model
    input_text, initial_state, final_state, probs = get_tensors(loaded_graph)

    # Sentences generation setup
    gen_sentences = [prime_word]
#     prev_state = sess.run(initial_state, {input_text: np.array([[1]])})
    prev_state = sess.run(initial_state, {input_text: np.array([[1]])})

    # Generate sentences
    for n in range(gen_length):
        # Dynamic Input
        dyn_input = [[vocab_to_int[word] for word in gen_sentences[-seq_length:]]]
#         print (dyn_input)
        dyn_seq_length = len(dyn_input[0])
#         print (dyn_seq_length)
        # Get Prediction
#         print (prev_state)
        probabilities, prev_state = sess.run([probs, final_state],{input_text: dyn_input, initial_state: prev_state})      
#         print (prev_state.shape)
#         print (probabilities)
        pred_word = pick_word(probabilities[0][dyn_seq_length-1], int_to_vocab)
        gen_sentences.append(pred_word)
    
    # Remove tokens
    gen_texts = ' '.join(gen_sentences)
    for key, token in token_dict.items():
        ending = ' ' if key in ['\n', '(', '"'] else ''
        gen_texts = gen_texts.replace(' ' + token.lower(), key)
    gen_texts = gen_texts.replace('\n ', '\n')
    gen_texts = gen_texts.replace('( ', '(')

    print(gen_texts)

INFO:tensorflow:Restoring parameters from ./save
Tensor("initial_state:0", shape=(1, 2, ?, 512), dtype=float32) Tensor("final_state:0", shape=(1, 2, ?, 512), dtype=float32)
world is fixes groups 1)'.

lighter to 1.13 an truth-conditions. 5.4711 happened. 4.026 the 6.241 saying that of
'(d 5.521 i after word, verb. a
sphere of 'and what this it: occurrence as the 5.123 of a 'is instance, is sum.
with a non-proposition as
5.47 it 2.034 brackets.-- it gathered prototype.) the fact that occam's
whatever are occam's 3.142 less 5.5352 in the our 6.43 if the articulate.

sole logical (written or described structures report in 6.341 exclude occurs. in the too that it seen
transition 6.1233 russell's i present. self logico-philosophicus as the express; deductions are rules: it is
a signified in the fact that the
('thing', bases. propositions are value. of the number', occur.

via the erect, solved by
expresses the gather of the pure signifies.

write the once shifting of the himself
affairs 'id

In [38]:
# print (int_to_vocab)

In [25]:
int_text

[13825,
 9343,
 4970,
 2887,
 12205,
 11137,
 11703,
 8429,
 14270,
 6656,
 1439,
 9343,
 7704,
 4342,
 4970,
 9738,
 213,
 9789,
 9343,
 3600,
 8931,
 12482,
 9343,
 3251,
 11259,
 9789,
 9343,
 2916,
 12064,
 10552,
 13145,
 4801,
 6285,
 9343,
 8972,
 11259,
 12064,
 10552,
 4772,
 2730,
 13145,
 5637,
 360,
 11225,
 9343,
 2916,
 12064,
 10552,
 13145,
 5637,
 360,
 13487,
 1519,
 1644,
 13087,
 9008,
 9343,
 8115,
 275,
 9343,
 4970,
 9738,
 213,
 9343,
 6019,
 9076,
 3865,
 10565,
 14732,
 12382,
 14744,
 9343,
 13936,
 10154,
 6776,
 13985,
 1819,
 12268,
 8016,
 12064,
 1897,
 7220,
 1819,
 12268,
 9343,
 825,
 12064,
 11914,
 4772,
 11021,
 7125,
 6367,
 12064,
 4241,
 8336,
 3360,
 13825,
 9343,
 4017,
 11589,
 4360,
 11137,
 5638,
 6849,
 2911,
 10209,
 4360,
 9122,
 7416,
 9343,
 2911,
 8484,
 9122,
 682,
 7818,
 2508,
 13205,
 9657,
 2911,
 9343,
 9642,
 6645,
 12268,
 1057,
 13873,
 12064,
 12268,
 11517,
 9343,
 13985,
 4360,
 12268,
 12350,
 7589,
 12064,
 1629,
 4241,
